# Music-Genre-Classifier Building Deep Learning Model

# 1. Import Dependencies 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import warnings
warnings.filterwarnings('ignore')

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
    
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Building Deep Neural Network for regression problem
import time 
import tensorflow as tf 
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from tensorflow.keras.layers import Dense, LSTM, BatchNormalization, Dropout, Input, Activation
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.activations import relu
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy

from sklearn.metrics import classification_report

# 2. Creating Callbacks

# 3. Building DCNN

# 4. Fitting and Tuning the Model

# 5. Exporting the model